<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Models" data-toc-modified-id="Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Models</a></span></li><li><span><a href="#Statistical-significance" data-toc-modified-id="Statistical-significance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Statistical significance</a></span></li></ul></div>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import linear_model
from sklearn import feature_selection
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
import sklearn as sklearn
import multiprocessing
import json
import nni

In [2]:
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.ensemble import RandomTreesEmbedding
from sklearn.neural_network import MLPRegressor
#from sklearn.linear_model import ElasticNet
from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import Imputer
from sklearn import metrics

In [3]:
import statsmodels.api as sm
from keras import optimizers
from sklearn.linear_model import SGDRegressor
#import statsmodels.api as sm

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1

In [5]:
import copy

# Models

In [6]:
# NN parameters
NUM_ITERATIONS = 300
BATCH_SIZE = 256
LEARNING_RATE = 0.001
LASSO_COEF = 50.0
DECAY_RATE = 0.0001

In [7]:
plt.style.use("ggplot")
n_cores = multiprocessing.cpu_count() # Getting the number of cores for multiprocessing

In [8]:
def get_gaussian_process_regressor():
    gp = GaussianProcessRegressor()
    return [gp], ['Gaussian Process']

In [9]:
def get_mlp_regressor(num_hidden_units=51):
    mlp = MLPRegressor(hidden_layer_sizes=num_hidden_units)
    return [mlp], ['Multi-Layer Perceptron']

In [10]:
def get_ensemble_models():
    grad = GradientBoostingRegressor(n_estimators=17, random_state=42, loss='lad', learning_rate=0.12, max_depth=10)
    classifier_list = [grad]
    classifier_name_list = ['Gradient Boost']
    return classifier_list, classifier_name_list

In [11]:
def print_evaluation_metrics(trained_model, trained_model_name, X_test, y_test):
    print('--------- For Model: ', trained_model_name, ' ---------\n')
    predicted_values = trained_model.predict(X_test)
    print("Mean absolute error: ",
          metrics.mean_absolute_error(y_test, predicted_values))
    print("Median absolute error: ",
          metrics.median_absolute_error(y_test, predicted_values))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_test, predicted_values))
    print("R2: ", metrics.r2_score(y_test, predicted_values))

In [12]:
def print_evaluation_metrics2(trained_model, trained_model_name, X_test, y_test):
    print('--------- For Model: ', trained_model_name, ' --------- (Train Data)\n')
    predicted_values = trained_model.predict(X_test)
    print("Mean absolute error: ",
          metrics.mean_absolute_error(y_test, predicted_values))
    print("Median absolute error: ",
          metrics.median_absolute_error(y_test, predicted_values))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_test, predicted_values))
    print("R2: ", metrics.r2_score(y_test, predicted_values))

In [13]:
def svm(X_train, y_train, X_val, y_val):
    model = SVR(gamma = 0.05, verbose = True) #was empty #0.1 #the - best gamma 0.05, c=0.5
    model.fit(X_train, y_train)
    print_evaluation_metrics(model, "svm", X_val, y_val.values.ravel())
    print_evaluation_metrics2(model, "svm", X_train, y_train.values.ravel())

In [14]:
def LinearModel(X_train, y_train, X_val, y_val):
    regr = linear_model.LinearRegression(n_jobs=int(0.8*n_cores)).fit(X_train, y_train)
    print_evaluation_metrics(regr, "linear model", X_val, y_val.values.ravel())
    print_evaluation_metrics2(regr, "linear model", X_train, y_train.values.ravel())

    return

In [15]:
def LinearModelRidge(X_train, y_train, X_val, y_val):
    regr = Ridge(alpha = 7) #7
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_val.values)

    print('Coefficients: \n', regr.coef_)
    print("Mean squared error: %.2f" % mean_squared_error(y_val, y_pred))
    print('Variance score: %.2f' % r2_score(y_val, y_pred))
    print("R2:", sklearn.metrics.r2_score(y_val, y_pred))

    print_evaluation_metrics(regr, "Linear Model Ridge", X_val, y_val)
    print_evaluation_metrics2(regr, "Linear Model Ridge", X_train, y_train)
    return 

In [16]:
def LinearModelLasso(X_train, y_train, X_val, y_val, alpha):
    regr = Lasso(alpha = alpha) #0.5
    regr.fit(X_train, y_train)

    print_evaluation_metrics(regr, "Linear Model Lasso", X_val, y_val)
    print_evaluation_metrics2(regr, "Linear Model Lasso", X_train, y_train)
    return

In [17]:
def simple_neural_network(X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(Dense(units=20, activation='relu', input_dim=len(X_train.values[0])))
    model.add(Dense(units=5, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    adam = optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=DECAY_RATE, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer=adam)
    model.fit(X_train, y_train, epochs=NUM_ITERATIONS, batch_size=BATCH_SIZE)
    print("finished fitting")
    print_evaluation_metrics(model, "NN", X_val, y_val)
    print_evaluation_metrics2(model, "NN", X_train, y_train)
    return 

In [18]:
def TreebasedModel(X_train, y_train, X_val, y_val):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    y_train = np.squeeze(y_train)
    y_val = np.squeeze(y_val)

    classifier_list, classifier_name_list = get_ensemble_models()
    for classifier, classifier_name in zip(classifier_list, classifier_name_list):
        classifier.fit(X_train, y_train)
        print_evaluation_metrics(classifier, classifier_name, X_val, y_val)
        print_evaluation_metrics2(classifier, classifier_name, X_train, y_train)
    return

In [19]:
def kmeans(X_train, y_train, X_val, y_val):
    n_clusters = 8
    #kmeans = KMeans(n_clusters=n_clusters, random_state=0, verbose=0, n_jobs=int(0.8*n_cores)).fit(X_train)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, verbose=0).fit(X_train)
    c_train = kmeans.predict(X_train)
    c_pred = kmeans.predict(X_val)
    centroids = kmeans.cluster_centers_
    y_val_stats = None
    predicted_values = None
    y_train_stats = None
    labels_stats = None
    for i in range(n_clusters):
        print('--------analyzing cluster %d--------' %i)
        train_mask = c_train==i
        std_train = np.std(y_train[train_mask])
        mean_train = np.mean(y_train[train_mask])
        print("# examples & price mean & std for training set within cluster %d is:(%d, %.2f, %.2f)" %(i, train_mask.sum(), np.float(mean_train), np.float(std_train)))
        pred_mask = c_pred==i
        std_pred = np.std(y_val[pred_mask])
        mean_pred = np.mean(y_val[pred_mask])
        print("# examples & price mean & std for validation set within cluster %d is:(%d, %.2f, %.2f)" %(i, pred_mask.sum(), np.float(mean_pred), np.float(std_pred)))
        if pred_mask.sum() == 0:
            print('Zero membered test set! Skipping the test and training validation.')
            continue
        #LinearModelRidge(X_train[train_mask], y_train[train_mask], X_val[pred_mask], y_val[pred_mask])
        regr = Ridge(alpha = 7) #7
        regr.fit(X_train[train_mask], y_train[train_mask])
        labels_pred = regr.predict(X_train[train_mask].values)
        y_pred = regr.predict(X_val[pred_mask].values)
        if (y_val_stats is None):
            y_val_stats = copy.deepcopy(y_val[pred_mask])
            y_train_stats = copy.deepcopy(y_train[train_mask])
            predicted_values = copy.deepcopy(y_pred)
            labels_stats = copy.deepcopy(labels_pred)

        else:
            y_val_stats = y_val_stats.append(y_val[pred_mask])
            y_train_stats = y_train_stats.append(y_train[train_mask])
            predicted_values = np.append(predicted_values, y_pred)
            labels_stats = np.append(labels_stats, labels_pred)
        print('--------Finished analyzing cluster %d--------' %i)
    print("Mean absolute error: ",
          metrics.mean_absolute_error(y_val_stats, predicted_values))
    print("Median absolute error: ",
          metrics.median_absolute_error(y_val_stats, predicted_values))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_val_stats, predicted_values))
    print("R2: ", metrics.r2_score(y_val_stats, predicted_values))
    print('------------TRAIN--------------------')
    print("Mean absolute error: ",
        metrics.mean_absolute_error(y_train_stats, labels_stats))
    print("Median absolute error: ",
        metrics.median_absolute_error(y_train_stats, labels_stats))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_train_stats, labels_stats))
    print("R2: ", metrics.r2_score(y_train_stats, labels_stats))


    return c_pred, centroids

In [20]:
def linear_model_SGD(X_train, y_train, X_val, y_val):
    model = SGDRegressor()
    model.fit(X_train, y_train)
    print_evaluation_metrics(model, "sgd", X_val, y_val.values.ravel())
    print_evaluation_metrics2(model, "sgd", X_train, y_train.values.ravel())

In [21]:
X_train = pd.read_csv('../Data/data_cleaned_train_comments_X.csv')
y_train = pd.read_csv('../Data/data_cleaned_train_y.csv')

X_val = pd.read_csv('../Data/data_cleaned_val_comments_X.csv')
y_val = pd.read_csv('../Data/data_cleaned_val_y.csv')

X_test = pd.read_csv('../Data/data_cleaned_test_comments_X.csv')
y_test = pd.read_csv('../Data/data_cleaned_test_y.csv')

In [22]:
s = True

In [23]:
if s == True:
    coeffs = np.load('../Data/selected_coefs.npy')
    col_set = set()
    cherry_picked_list = [
    'host_identity_verified',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'guests_included',
    'security_deposit',
    'cleaning_fee',
    'extra_people',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'comments',

    ]
    
    for i in range(len(coeffs)):
        if (coeffs[i]):
            col_set.add(X_train.columns[i])
    X_train = X_train[list(col_set)]
    X_val = X_val[list(col_set)]
    X_test = X_test[list(col_set)]

    X_concat = pd.concat([X_train, X_val], ignore_index=True)
    y_concat = pd.concat([y_train, y_val], ignore_index=True)

In [ ]:
if __name__ == "__main__":

    #coeffs = np.load('../Data/selected_coefs_pvals.npy')
    coeffs = np.load('../Data/selected_coefs.npy')
    col_set = set()
    cherry_picked_list = [
    'host_identity_verified',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'guests_included',
    'security_deposit',
    'cleaning_fee',
    'extra_people',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'comments',

    ]
    
    for i in range(len(coeffs)):
        if (coeffs[i]):
            col_set.add(X_train.columns[i])
    X_train = X_train[list(col_set)]
    X_val = X_val[list(col_set)]
    X_test = X_test[list(col_set)]

    X_concat = pd.concat([X_train, X_val], ignore_index=True)
    y_concat = pd.concat([y_train, y_val], ignore_index=True)

    #RUN WITHOUT FEATURE SELECTION FOR THE BASELINE
    """
    print("--------------------Linear Regression--------------------")
    LinearModel(X_concat, y_concat, X_test, y_test)
    """

    print("--------------------Tree-based Model--------------------")
    TreebasedModel(X_concat, y_concat, X_test, y_test)
    print("--------------------KMeans Clustering--------------------")
    c_pred, centroids = kmeans(X_concat, y_concat, X_test, y_test)




    print("--------------------------------------------------")
    LinearModelRidge(X_concat, y_concat, X_test, y_test)
    print("--------------------------------------------------")

    print("--------------------------------------------------")
    LinearModelLasso(X_train, y_train, X_val, y_val, 0.002)
    print("--------------------------------------------------")
    simple_neural_network(X_concat, y_concat, X_test, y_test)

    svm(X_concat, y_concat, X_test, y_test)

--------------------Tree-based Model--------------------


/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:301: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_error' which is equivalent.
  warnings.warn(


--------- For Model:  Gradient Boost  ---------

Mean absolute error:  0.3221680756948255
Median absolute error:  0.25290180006538043
Mean squared error:  0.19082347147345055
R2:  0.5983872008849019
--------- For Model:  Gradient Boost  --------- (Train Data)

Mean absolute error:  0.23047799483672443
Median absolute error:  0.15919804894709566
Mean squared error:  0.12478578363199358
R2:  0.7396295916010915
--------------------KMeans Clustering--------------------
--------analyzing cluster 0--------
# examples & price mean & std for training set within cluster 0 is:(3482, 5.06, 0.62)
# examples & price mean & std for validation set within cluster 0 is:(438, 5.06, 0.61)
--------Finished analyzing cluster 0--------
--------analyzing cluster 1--------
# examples & price mean & std for training set within cluster 1 is:(5115, 4.68, 0.61)
# examples & price mean & std for validation set within cluster 1 is:(589, 4.64, 0.58)
--------Finished analyzing cluster 1--------
--------analyzing clus

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1423/2739673014.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print("# examples & price mean & std for training set within cluster %d is:(%d, %.2f, %.2f)" %(i, train_mask.sum(), np.float(mean_train), np.float(std_train)))
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbd

/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1423/2739673014.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train_stats = y_train_stats.append(y_train[train_mask])
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1423/2739673014.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and g

...

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/300
112/112 [==============================] - 1s 2ms/step - loss: 3.2399
Epoch 2/300
112/112 [==============================] - 0s 3ms/step - loss: 0.5670
Epoch 3/300
112/112 [==============================] - 0s 2ms/step - loss: 0.3190
Epoch 4/300
112/112 [==============================] - 0s 3ms/step - loss: 0.2592
Epoch 5/300
112/112 [==============================] - 0s 3ms/step - loss: 0.2304
Epoch 6/300
112/112 [==============================] - 0s 3ms/step - loss: 0.2126
Epoch 7/300
112/112 [==============================] - 0s 3ms/step - loss: 0.2000
Epoch 8/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1913
Epoch 9/300
112/112 [==============================] - 1s 5ms/step - loss: 0.1847
Epoch 10/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1790
Epoch 11/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1755
Epoch 12/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1717
Epoch 13/300


112/112 [==============================] - 0s 2ms/step - loss: 0.1211
Epoch 197/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1210
Epoch 198/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1210
Epoch 199/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1212
Epoch 200/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1207
Epoch 201/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1205
Epoch 202/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1210
Epoch 203/300
112/112 [==============================] - 0s 2ms/step - loss: 0.1208
Epoch 204/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1206
Epoch 205/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1210
Epoch 206/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1205
Epoch 207/300
112/112 [==============================] - 0s 3ms/step - loss: 0.1204
Epoch 

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


.........................

# Statistical significance

In [70]:
def xg_boost_sig(X, y, k=5):
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    import xgboost as xgb
    model = xgb.XGBRegressor(colsample_bytree = 0.8, n_estimators=150, random_state=42, loss='lad', learning_rate=0.05, max_depth=6, subsample=0.8)

    kf = KFold(n_splits=k)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))

    return

In [54]:
def LinearModel_sig(X, y, k=5):
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    model = linear_model.LinearRegression(n_jobs=int(0.8*n_cores))
    kf = KFold(n_splits=k)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))

    return

In [56]:
def LinearModelRidge_sig(X, y,k=5):
    model = Ridge(alpha = 7) #7
    kf = KFold(n_splits=k)
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [58]:
def LinearModelLasso_sig(X,y, k=5, alpha=0.002):
    model = Lasso(alpha = alpha) #0.5
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    kf = KFold(n_splits=k)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [68]:
def simple_neural_network_sig(X, y, k=5):
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    kf = KFold(n_splits=k)
    model = Sequential()
    model.add(Dense(units=20, activation='relu', input_dim=len(X.values[0])))
    model.add(Dense(units=5, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    adam = optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=DECAY_RATE, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer=adam)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train,epochs=NUM_ITERATIONS, batch_size=BATCH_SIZE)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [65]:
def TreebasedModel_sig(X,y,k=5):
    kf = KFold(n_splits=k)
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    classifier_list, classifier_name_list = get_ensemble_models()
    for classifier, classifier_name in zip(classifier_list, classifier_name_list):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.loc[train_index], X.loc[test_index] 
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            classifier.fit(X_train, y_train)
            predicted_values = classifier.predict(X_test)
            scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
            scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
            scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [67]:
def linear_model_SGD_sig(X,y,k=5):
    kf = KFold(n_splits=k)
    model = SGDRegressor()
    scores_r2=[]
    scores_mae = []
    scores_mse = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [30]:
def svm_significance(X, y, k=5):
    kf = KFold(n_splits=k)
    scores = []
    model = SVR(gamma = 0.05, verbose = True)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index] 
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        scores_mae.append(metrics.mean_absolute_error(y_test, predicted_values))
        scores_mse.append(metrics.mean_squared_error(y_test, predicted_values))
        scores_r2.append(metrics.r2_score(y_test, predicted_values))

    mean_score_r2 = sum(scores_r2) / len(scores_r2)
    st_dev_r2=np.std(scores_r2)
    
    print('R2 mean:  ' + str(mean_score_r2))
    print('R2 std:  ' + str(st_dev_r2))
    
    mean_score_mae = sum(scores_mae) / len(scores_mae)
    st_dev_mae=np.std(scores_mae)
    
    print('MAE mean:  ' + str(mean_score_mae))
    print('MAE std:  ' + str(st_dev_mae))
    
    mean_score_mse = sum(scores_mse) / len(scores_mse)
    st_dev_mse=np.std(scores_mse)
    
    print('MSE mean:  ' + str(mean_score_mse))
    print('MSE std:  ' + str(st_dev_mse))
    return

In [72]:
X_concat_old = X_concat.drop('EDU10_E', axis = 1)

In [73]:
xg_new = xg_boost_sig(X_concat, y_concat, k=5)
xg_old = xg_boost_sig(X_concat_old, y_concat, k=5)

[22:25:26] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "loss" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:25:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "loss" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:26:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_6

In [24]:
from sklearn.model_selection import KFold

In [74]:
lin_new = LinearModel_sig(X_concat, y_concat, k=5)
lin_old = LinearModel_sig(X_concat_old, y_concat, k=5)

R2 mean:  0.6817847785462273
R2 std:  0.00842797137238802
MAE mean:  0.27792521383523955
MAE std:  0.002605091912297446
MSE mean:  0.15256878129800153
MSE std:  0.007716869887086794
R2 mean:  0.6796225807183897
R2 std:  0.008393974102545703
MAE mean:  0.2792111701697516
MAE std:  0.002842824518554625
MSE mean:  0.15361048532357274
MSE std:  0.007866634858221163


In [75]:
linr_new = LinearModelRidge_sig(X_concat, y_concat,k=5)
linr_old = LinearModelRidge_sig(X_concat_old, y_concat,k=5)

R2 mean:  0.6816042668907638
R2 std:  0.00897117842417922
MAE mean:  0.2783543233350677
MAE std:  0.002936122547603586
MSE mean:  0.1526566262990877
MSE std:  0.00787518067441251
R2 mean:  0.6794035330188143
R2 std:  0.008891681666679406
MAE mean:  0.2796390142212753
MAE std:  0.003099758022632467
MSE mean:  0.15371770001510723
MSE std:  0.008030407525714447


In [76]:
linl_new  =LinearModelLasso_sig(X_concat,y_concat, alpha=0.002)
linl_old  =LinearModelLasso_sig(X_concat_old,y_concat, alpha=0.002)

R2 mean:  0.6542038656537376
R2 std:  0.009548471106548558
MAE mean:  0.29107077894869093
MAE std:  0.00355834447276121
MSE mean:  0.16578766504390136
MSE std:  0.008359527097508451
R2 mean:  0.644905400434026
R2 std:  0.009165985068889536
MAE mean:  0.29602419518906886
MAE std:  0.003870198777560502
MSE mean:  0.17025416947019967
MSE std:  0.00862931834984352


In [79]:
nn_new = simple_neural_network_sig(X_concat, y_concat, k=5)
nn_old = simple_neural_network_sig(X_concat_old, y_concat, k=5)

Epoch 1/300


/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


89/89 [==============================] - 1s 2ms/step - loss: 12.8322
Epoch 2/300
89/89 [==============================] - 0s 3ms/step - loss: 1.3552
Epoch 3/300
89/89 [==============================] - 0s 2ms/step - loss: 0.8650
Epoch 4/300
89/89 [==============================] - 0s 3ms/step - loss: 0.6670
Epoch 5/300
89/89 [==============================] - 0s 3ms/step - loss: 0.5451
Epoch 6/300
89/89 [==============================] - 0s 3ms/step - loss: 0.4600
Epoch 7/300
89/89 [==============================] - 0s 2ms/step - loss: 0.4000
Epoch 8/300
89/89 [==============================] - 0s 2ms/step - loss: 0.3577
Epoch 9/300
89/89 [==============================] - 0s 2ms/step - loss: 0.3271
Epoch 10/300
89/89 [==============================] - 0s 2ms/step - loss: 0.3043
Epoch 11/300
89/89 [==============================] - 0s 2ms/step - loss: 0.2867
Epoch 12/300
89/89 [==============================] - 0s 2ms/step - loss: 0.2728
Epoch 13/300
89/89 [============================

89/89 [==============================] - 0s 3ms/step - loss: 0.1317
Epoch 103/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1310
Epoch 104/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1307
Epoch 105/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1303
Epoch 106/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1300
Epoch 107/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1300
Epoch 108/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1296
Epoch 109/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1291
Epoch 110/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1291
Epoch 111/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1286
Epoch 112/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1283
Epoch 113/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1280
Epoch 114/300
89/89 [=========

89/89 [==============================] - 0s 2ms/step - loss: 0.1147
Epoch 203/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1145
Epoch 204/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1143
Epoch 205/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1146
Epoch 206/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1144
Epoch 207/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1140
Epoch 208/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1142
Epoch 209/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1140
Epoch 210/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1140
Epoch 211/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1139
Epoch 212/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1138
Epoch 213/300
89/89 [==============================] - 0s 2ms/step - loss: 0.1136
Epoch 214/300
89/89 [=========

...

89/89 [==============================] - 0s 4ms/step - loss: 0.1041
Epoch 202/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1040
Epoch 203/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1041
Epoch 204/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1040
Epoch 205/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1041
Epoch 206/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1039
Epoch 207/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1039
Epoch 208/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1040
Epoch 209/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1039
Epoch 210/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1039
Epoch 211/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1041
Epoch 212/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1040
Epoch 213/300
89/89 [=========

178/178 [==============================] - 1s 4ms/step
Epoch 1/300
89/89 [==============================] - 1s 10ms/step - loss: 0.1106
Epoch 2/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1101
Epoch 3/300
89/89 [==============================] - 1s 6ms/step - loss: 0.1099
Epoch 4/300
89/89 [==============================] - 1s 7ms/step - loss: 0.1096
Epoch 5/300
89/89 [==============================] - 1s 8ms/step - loss: 0.1093
Epoch 6/300
89/89 [==============================] - 1s 7ms/step - loss: 0.1092
Epoch 7/300
89/89 [==============================] - 1s 6ms/step - loss: 0.1090
Epoch 8/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1089
Epoch 9/300
89/89 [==============================] - 1s 7ms/step - loss: 0.1088
Epoch 10/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1087
Epoch 11/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1087
Epoch 12/300
89/89 [==============================] - 0s 5ms/s

89/89 [==============================] - 0s 4ms/step - loss: 0.1055
Epoch 102/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1053
Epoch 103/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1056
Epoch 104/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1054
Epoch 105/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1056
Epoch 106/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1054
Epoch 107/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1055
Epoch 108/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1056
Epoch 109/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1053
Epoch 110/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1054
Epoch 111/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1053
Epoch 112/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1053
Epoch 113/300
89/89 [=========

89/89 [==============================] - 0s 3ms/step - loss: 0.1043
Epoch 202/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1043
Epoch 203/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1044
Epoch 204/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1043
Epoch 205/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1042
Epoch 206/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1043
Epoch 207/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1042
Epoch 208/300
89/89 [==============================] - 0s 3ms/step - loss: 0.1042
Epoch 209/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1042
Epoch 210/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1042
Epoch 211/300
89/89 [==============================] - 0s 4ms/step - loss: 0.1043
Epoch 212/300
89/89 [==============================] - 0s 5ms/step - loss: 0.1043
Epoch 213/300
89/89 [=========

178/178 [==============================] - 1s 3ms/step
R2 mean:  0.7145523722214202
R2 std:  0.039906240898503156
MAE mean:  0.26567856179425575
MAE std:  0.014284842012995439
MSE mean:  0.13678286810373047
MSE std:  0.019178295688837235


In [77]:
gb_new = TreebasedModel_sig(X_concat,y_concat)
gb_old = TreebasedModel_sig(X_concat_old,y_concat)

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:301: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_error' which is equivalent.
  warnings.warn(
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:301: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_err

R2 mean:  0.6696347030650347
R2 std:  0.012242512532241848
MAE mean:  0.27616456031329883
MAE std:  0.0036469820398265314
MSE mean:  0.158444678233524
MSE std:  0.009853195711449625


/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:301: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_error' which is equivalent.
  warnings.warn(
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:301: FutureWarning: The loss 'lad' was deprecated in v1.0 and will be removed in version 1.2. Use 'absolute_err

R2 mean:  0.6640998585956381
R2 std:  0.010339874392652803
MAE mean:  0.27958880253040563
MAE std:  0.003219768610871768
MSE mean:  0.16107395196596808
MSE std:  0.008978664103269533


In [78]:
lsgd_new = linear_model_SGD_sig(X_concat,y_concat,k=5)
lsgd_old = linear_model_SGD_sig(X_concat_old,y_concat,k=5)

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was 

R2 mean:  0.6569772889235173
R2 std:  0.007978676526743342
MAE mean:  0.28858794463673026
MAE std:  0.0031302349143083456
MSE mean:  0.16444542252639494
MSE std:  0.007655504820401673


/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was 

R2 mean:  0.6514423255723344
R2 std:  0.008969155753747867
MAE mean:  0.29254945365215335
MAE std:  0.004522404656886578
MSE mean:  0.1671300762281714
MSE std:  0.008684651221101585


In [ ]:
svm_new = svm_significance(X_concat, y_concat, k=5)
svm_old = svm_significance(X_concat_old, y_concat, k=5)

/Users/yousseflahlou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]........